# Script to download Sentinel2 Landsat and ArcticDEM Data form GEE

In [1]:
import geemap
import geopandas as gpd
from pathlib import Path
import ee
import geopandas as gpd

ee.Initialize()

In [2]:
LOCATION = 'Bykovsky'
vector = f'{LOCATION}/{LOCATION}_AOI_2500.geojson'
aoi_fc = geemap.vector_to_ee(vector)
crs = 'EPSG:32652'

In [121]:
image_date = '2021-07-21'
s2_export_bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','B8','B8A', 'B11', 'B12']
l8_export_bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8']

### S2 - Select S2 images

In [31]:
s2_col = ee.ImageCollection("COPERNICUS/S2").filterBounds(aoi_fc).filterDate(image_date, '2021-07-22')

In [54]:
# Number of intersecting images
len(s2_col.toList(10).getInfo())

2

#### Select correct image

In [69]:
s2_image = ee.Image(s2_col.toList(10).get(1))

In [94]:
Map = geemap.Map()
Map.add_ee_layer(s2_image, vis_params=dict(min=0, max=2000))
Map.add_ee_layer(aoi_fc)
Map.centerObject(s2_image)
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

#### Download S2 image

In [122]:
image_id = s2_image.id().getInfo()
geemap.ee_export_image(ee_object=s2_image.select(s2_export_bands), filename=f'{LOCATION}/S2_{image_id}.tif', region=aoi_fc.geometry(), scale=10)

Generating URL ...
Please wait ...
Data downloaded to C:\Users\initze\OneDrive\146_RTSIn_ActionGroup\01_digitization_Experiment\Bykovsky\S2_20210721T035541_20210721T035540_T52WEE.tif


### Download Arctic DEM

In [92]:
arcticDEM = ee.Image("UMN/PGC/ArcticDEM/V3/2m_mosaic")

In [95]:
geemap.ee_export_image(ee_object=arcticDEM, filename=f'{LOCATION}/ArcticDEM.tif', region=aoi_fc.geometry(), scale=2, crs='EPSG:32652')

Generating URL ...
Please wait ...
Data downloaded to C:\Users\initze\OneDrive\146_RTSIn_ActionGroup\01_digitization_Experiment\Bykowsky\ArcticDEM.tif


In [107]:
L8_col = ee.ImageCollection("LANDSAT/LC08/C02/T1_TOA").filterBounds(aoi_fc).filterDate('2021-07-18', '2021-07-23')

In [108]:
len(L8_col.toList(10).getInfo())
#L8_col.getInfo()

3

In [113]:
L8_image = ee.Image(L8_col.toList(10).get(1))

In [114]:
Map = geemap.Map()
Map.add_ee_layer(L8_image, vis_params=dict(min=0, max=0.2))
Map.add_ee_layer(aoi_fc)
Map.centerObject(L8_image)
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [124]:
image_id = L8_image.id().getInfo()
geemap.ee_export_image(ee_object=L8_image.select(l8_export_bands), filename=f'{LOCATION}/L8_{image_id}.tif', region=aoi_fc.geometry(), scale=15)

Generating URL ...
Please wait ...
Data downloaded to C:\Users\initze\OneDrive\146_RTSIn_ActionGroup\01_digitization_Experiment\Bykovsky\L8_LC08_198234_20210718.tif


### Create empty shapefile

In [6]:
gdf = gpd.GeoDataFrame(columns=['id'], crs=crs )

In [7]:
gdf.to_file(f'{LOCATION}/digitized_polygon.shp')

ValueError: Cannot write empty DataFrame to file.